# Bscktest for Regulation method

In [1]:
import numpy as np
import pandas as pd
import time
from scipy.linalg import solve, kron
from scipy import stats
from scipy.stats import norm
import math
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

## Paramters

In [2]:
wsdata = './'
inputfile = 'daily_smith_wilson_20000101_.csv'

In [3]:
# initial parameters
para0 = np.concatenate((
    np.array([0.17]),
    np.array([1.7000000e-02,  2.3000000e-02,
        2.8000000e-02,  3.3000000e-02,  3.9000000e-02,  3.8000000e-02,
        3.8000000e-02,  3.9000000e-02,  4.0000000e-02,  5.0000000e-02,
        2.8000000e-01,  6.2000000e-01]),
    np.array([9.9648000e-01,  9.9603000e-01,9.9667000e-01]),
    np.array([3.1690000e+00, -3.2460000e+00, -3.5640000e+00]),
    np.array([1.0720000e-01, -1.0580000e-01,  1.2300000e-02, -1.1060000e-01,-3.1400000e-02,  7.7500000e-02,])
))


# parameter update scale
paraStep = np.concatenate((
    np.array([-30]),
    np.repeat(-3, 10),
    np.repeat(-2, 2),
    np.repeat(-4, 3),
    np.repeat(-2, 3),
    np.repeat(-3, 6)
))
# grid：Nelson–Siegel grids
grid = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30])
gridn = len(grid)
lambdastep = np.arange(0.16, 0.2 + 0.01, 0.01)
maxstepn = 10000
tstep = 52

maturities = ['1','2','3','4','5','6','7','8','9','10','20','30']
maturities_float = np.array([float(m) for m in maturities])
maturities2 = np.arange(1, 51) # for generating yield curves from the calibrated parameters
maturities2_float = np.array([float(m) for m in maturities2])
initial_lambdastep = np.arange(0.16, 0.2, 0.1)

## Helper functions

In [4]:
# Smith-wilson interpolation
import pandas as pd
import numpy as np
import math
from IPython.display import display

def hh(z: float) -> float:
    return (z + math.exp(-z)) / 2.0

def Hmat(u: float, v: float) -> float:
    return hh(u + v) - hh(abs(u - v))

def Galfa(alfa: float, Q: np.ndarray, mm: int, umax: int, nrofcoup: int, T2: int, Tau: float):
    size = umax * nrofcoup
    h_mat = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            u_i = alfa * (i + 1) / nrofcoup
            u_j = alfa * (j + 1) / nrofcoup
            h_mat[i, j] = Hmat(u_i, u_j)
    temp1 = []
    for i in range(mm):
        temp1.append(1.0 - np.sum(Q[i, :]))
    temp1 = np.array(temp1).reshape(mm, 1)
    
    Q_h_Qt = Q @ h_mat @ Q.T  # (mm x mm)行列
    inv_Q_h_Qt = np.linalg.inv(Q_h_Qt)
    b = inv_Q_h_Qt @ temp1
    gamma = Q.T @ b
    gamma = gamma.ravel()
    
    temp2 = 0.0
    temp3 = 0.0
    for i in range(size):
        t = (i + 1) / nrofcoup
        temp2 += gamma[i] * t
        temp3 += gamma[i] * math.sinh(alfa * t)
    kappa = (1.0 + alfa * temp2) / temp3
    galfa_val = alfa / abs(1.0 - kappa * math.exp(T2 * alfa))
    return (galfa_val - Tau, gamma)

def AlfaScan(lastalfa: float, stepsize: float, Q: np.ndarray, mm: int, umax: int,
             nrofcoup: int, T2: int, Tau: float):
    scan_start = lastalfa + stepsize/10.0 - stepsize
    scan_end = lastalfa
    scan_step = stepsize / 10.0
    alphas = np.arange(scan_start, scan_end + 1e-12, scan_step)
    for alfa in alphas:
        val, gamma = Galfa(alfa, Q, mm, umax, nrofcoup, T2, Tau)
        if val <= 0:
            return (alfa, gamma)
    return (lastalfa, gamma)

def smith_wilson_brute_force(Instrument: str, data_in: np.ndarray,
                              nrofcoup: int, CRA: float, UFRac: float,
                              alfamin: float, Tau: float, T2: int):
    Tau = Tau / 10000.0

    # data_in[:,0]: Indicator, data_in[:,1]: Maturity, data_in[:,2]: Rate
    indicators = data_in[:, 0]
    mm = int(np.sum(indicators))
    u = []
    r = []
    for row in data_in:
        if row[0] == 1:
            maturity = row[1]
            rate = row[2]
            adj_rate = rate - CRA / 10000.0
            u.append(maturity)
            r.append(adj_rate)
    u = np.array(u, dtype=int)
    r = np.array(r, dtype=float)
    umax = np.max(u)
    lnUFR = math.log(1.0 + UFRac)
    
    if Instrument.lower() == "zero":
        nc = 1
    else:
        nc = nrofcoup

    Q = np.zeros((mm, umax * nc))
    if Instrument.lower() == "zero":
        for i in range(mm):
            maturity_i = u[i]
            rate_i = r[i]
            col_idx = maturity_i - 1
            Q[i, col_idx] = math.exp(-lnUFR * maturity_i) * ((1.0 + rate_i)**maturity_i)
    else:
        for i in range(mm):
            maturity_i = u[i]
            rate_i = r[i]
            for j in range(1, maturity_i * nc):
                col_idx = j - 1
                Q[i, col_idx] = math.exp(-lnUFR * (j / nc)) * (rate_i / nc)
            col_idx = (maturity_i * nc) - 1
            Q[i, col_idx] = math.exp(-lnUFR * maturity_i) * (1.0 + rate_i / nc)
    
    # compute the optimal alpha
    val_min, gamma_min = Galfa(alfamin, Q, mm, umax, nc, T2, Tau)
    if val_min <= 0.0:
        alfa_opt = alfamin
        gamma_opt = gamma_min
    else:
        stepsize = 0.1
        alfa_candidate = alfamin
        found = False
        for alfa in np.arange(alfamin + stepsize, 20.0 + 1e-12, stepsize):
            val, gamma_ = Galfa(alfa, Q, mm, umax, nc, T2, Tau)
            if val <= 0.0:
                alfa_candidate = alfa
                gamma_opt = gamma_
                found = True
                break
        if not found:
            alfa_candidate = 20.0
            _, gamma_opt = Galfa(alfa_candidate, Q, mm, umax, nc, T2, Tau)
        precision = 6
        for _ in range(1, precision):
            alfa_candidate, gamma_opt = AlfaScan(alfa_candidate, stepsize, Q, mm, umax, nc, T2, Tau)
            stepsize /= 10.0
        alfa_opt = alfa_candidate

    size = umax * nc
    h_mat = np.zeros((122, size))
    g_mat = np.zeros((122, size))
    for i in range(122):
        for j in range(size):
            h_mat[i, j] = Hmat(alfa_opt * i, alfa_opt * (j + 1) / nc)
            if ((j + 1) / nc) > i:
                g_mat[i, j] = alfa_opt * (1.0 - math.exp(-alfa_opt * (j + 1) / nc) * math.cosh(alfa_opt * i))
            else:
                g_mat[i, j] = alfa_opt * math.exp(-alfa_opt * i) * math.sinh(alfa_opt * (j + 1) / nc)
    
    tempdiscount = (h_mat @ gamma_opt.reshape(-1, 1)).ravel()
    tempintensity = (g_mat @ gamma_opt.reshape(-1, 1)).ravel()
    
    discount = np.zeros(122)
    fwintensity = np.zeros(122)
    yldintensity = np.zeros(122)
    zeroac = np.zeros(122)
    forwardac = np.zeros(122)
    forwardcc = np.zeros(122)
    zerocc = np.zeros(122)
    
    # i=0
    temp_sum = 0.0
    for j in range(size):
        t = (j + 1) / nc
        temp_sum += (1.0 - math.exp(-alfa_opt * t)) * gamma_opt[j]
    yldintensity[0] = lnUFR - alfa_opt * temp_sum
    fwintensity[0] = yldintensity[0]
    discount[0] = 1.0

    # i=1
    if 1.0 + tempdiscount[1] > 0:
        yldintensity[1] = lnUFR - math.log(1.0 + tempdiscount[1])
        discount[1] = math.exp(-lnUFR) * (1.0 + tempdiscount[1])
    else:
        yldintensity[1] = np.nan
        discount[1] = np.nan

    if discount[1] and discount[1] != 0:
        zeroac[1] = 1.0 / discount[1] - 1.0
    else:
        zeroac[1] = np.nan
    fwintensity[1] = lnUFR - (tempintensity[1] / (1.0 + tempdiscount[1]) if (1.0 + tempdiscount[1]) > 0 else np.nan)
    forwardac[1] = zeroac[1]

    # i = 2～120
    for i in range(2, 121):
        if 1.0 + tempdiscount[i] > 0:
            yldintensity[i] = lnUFR - math.log(1.0 + tempdiscount[i]) / i
        else:
            yldintensity[i] = np.nan
        fwintensity[i] = lnUFR - (tempintensity[i] / (1.0 + tempdiscount[i]) if (1.0 + tempdiscount[i]) > 0 else np.nan)
        discount[i] = math.exp(-lnUFR * i) * (1.0 + tempdiscount[i])
        if discount[i] > 0:
            zeroac[i] = (1.0 / discount[i]) ** (1.0 / i) - 1.0
        else:
            zeroac[i] = np.nan
        forwardac[i] = (discount[i - 1] / discount[i] - 1.0) if discount[i] != 0 else np.nan

    discount[121] = alfa_opt
    yldintensity[121] = 0.0
    fwintensity[121] = 0.0
    zeroac[121] = 0.0
    forwardac[121] = 0.0

    for i in range(1, 121):
        if 1.0 + forwardac[i] > 0:
            forwardcc[i] = math.log(1.0 + forwardac[i])
        else:
            forwardcc[i] = np.nan
        if 1.0 + zeroac[i] > 0:
            zerocc[i] = math.log(1.0 + zeroac[i])
        else:
            zerocc[i] = np.nan

    result = np.column_stack([discount, yldintensity, zeroac, fwintensity, forwardcc, forwardac])
    return result

def run_on_dataframe(df: pd.DataFrame):
    maturity_cols = [col for col in df.columns if col != 'Date']
    results = []
    for idx, row in df.iterrows():
        date = row['Date']
        input_list = []
        for col in maturity_cols:
            rate = row[col]
            if not pd.isna(rate):
                try:
                    maturity = int(col)
                except ValueError:
                    maturity = float(col)
                input_list.append([1, maturity, float(rate)])
        if len(input_list) == 0:
            continue
        data_in = np.array(input_list)
        Instrument = "Bond"
        nrofcoup = 2
        CRA = 0
        UFRac = 0.025
        alfamin = 0.05
        Tau = 1.0
        T2 = 60
        
        sw_result = smith_wilson_brute_force(Instrument, data_in, nrofcoup, CRA, UFRac, alfamin, Tau, T2)
        optimal_alpha = sw_result[-1, 0]
        spot_rates = {}
        for i in range(1, 121):
            spot_rates[str(i)] = sw_result[i, 2]
        spot_rates["optimal_alpha"] = optimal_alpha
        row_result = {"Date": date}
        row_result.update(spot_rates)
        results.append(row_result)
    cols = ["Date"] + [str(i) for i in range(1, 121)] + ["optimal_alpha"]
    return pd.DataFrame(results, columns=cols)

In [ ]:
# utility functions
def lyapunov(N, phi, Q):
    I = np.eye(N * N)
    K_mat = np.kron(phi, phi)
    vecP = np.linalg.solve(I - K_mat, Q.flatten())
    return vecP.reshape(N, N)

def Nelson_Siegel_factor_loadings(l, m):
    m = np.array(m)
    column1 = np.ones(len(m))
    column2 = (1 - np.exp(-l * m)) / (l * m)
    column3 = column2 - np.exp(-l * m)
    lambmat = np.column_stack((column1, column2, column3))
    return lambmat

def Kfilter(logLik, N, T, Y, Z, a_t, P_t, H, a_tt, P_tt, v2, v1, phi, mu, Q, prev, M, Yf, lik):
    for t in range(T):
        v = Y[t, :] - Z @ a_t[t, :]
        F = Z @ P_t[t, :, :] @ Z.T + H
        detF = np.linalg.det(F)
        if detF <= 1e-30 or np.isnan(detF) or np.isinf(detF):
            logLik = -10**15
            break
        else:
            F_inv = np.linalg.inv(F)
            logLik = logLik - 0.5 * (np.log(detF) + v.T @ F_inv @ v)
        a_tt[t, :] = a_t[t, :] + P_t[t, :, :] @ Z.T @ F_inv @ v
        P_tt[t, :, :] = P_t[t, :, :] - P_t[t, :, :] @ Z.T @ F_inv @ Z @ P_t[t, :, :]
        v1[t, :] = Z @ a_tt[t, :]
        v2[t, :] = Y[t, :] - Z @ a_tt[t, :]
        a_t[t+1, :] = phi @ a_tt[t, :] + (np.eye(N) - phi) @ mu.flatten()
        P_t[t+1, :, :] = phi @ P_tt[t, :, :] @ phi.T + Q

    if prev:
        t0 = T - 1
        for m_i in range(1, M+1):
            Yf[t0 + m_i, :] = Z @ a_t[t0 + m_i, :]
            a_tt[t0 + m_i, :] = a_t[t0 + m_i, :]
            P_tt[t0 + m_i, :, :] = P_t[t0 + m_i, :, :]
            a_t[t0 + m_i + 1, :] = phi @ a_tt[t0 + m_i, :] + (np.eye(N) - phi) @ mu.flatten()
            P_t[t0 + m_i + 1, :, :] = phi @ P_tt[t0 + m_i, :, :] @ phi.T + Q

    if lik:
        return -logLik
    else:
        return {'a_tt': a_tt, 'a_t': a_t, 'P_tt': P_tt, 'P_t': P_t, 'v2': v2, 'v1': v1, 'Yf': Yf}

def kalman(para, Y, lik, prev, ahead, grid):
    l = para[0]
    m = grid
    mlen = len(m)
    M = ahead
    if prev:
        T = Y.shape[0]
        Yf = Y.copy()
        Yf[T-M:T, :] = np.nan
        Y = Y[:T-M, :]
        T = Y.shape[0]
    else:
        T = Y.shape[0]
        Yf = None
    
    pars = {}
    W = Y.shape[1]
    N = 3
    pars['mu'] = np.full((N, 1), np.nan)
    pars['phi'] = np.eye(N)
    pars['H'] = np.eye(W)
    pars['Q'] = np.eye(N)
    
    # Loading matrix
    pars['Z'] = Nelson_Siegel_factor_loadings(l, m)
    
    for i in range(mlen):
        pars['H'][i, i] = para[1 + i]
    H = pars['H'] ** 2
    
    # VAR(1) coefficient matrix
    pars['phi'][0, 0] = para[mlen+1]
    pars['phi'][0, 1] = 0
    pars['phi'][0, 2] = 0
    pars['phi'][1, 0] = 0
    pars['phi'][1, 1] = para[mlen+2]
    pars['phi'][1, 2] = 0
    pars['phi'][2, 0] = 0
    pars['phi'][2, 1] = 0
    pars['phi'][2, 2] = para[mlen+3]
    
    pars['mu'][0, 0] = para[mlen+4]
    pars['mu'][1, 0] = para[mlen+5]
    pars['mu'][2, 0] = para[mlen+6]
    
    pars['Q'][0, 0] = para[mlen+7]
    pars['Q'][1, 0] = para[mlen+8]
    pars['Q'][1, 1] = para[mlen+9]
    pars['Q'][2, 0] = para[mlen+10]
    pars['Q'][2, 1] = para[mlen+11]
    pars['Q'][2, 2] = para[mlen+12]
    
    Q = pars['Q'] @ pars['Q'].T
    
    # Initialize matrices
    if prev:
        a_t = np.full((T+M, N), np.nan)
        a_tt = np.full((T, N), np.nan)
        P_t = np.full((T+M, N, N), np.nan)
        P_tt = np.full((T, N, N), np.nan)
    else:
        a_t = np.full((T+1, N), np.nan)
        a_tt = np.full((T, N), np.nan)
        P_t = np.full((T+1, N, N), np.nan)
        P_tt = np.full((T, N, N), np.nan)
    v1 = np.full((T, W), np.nan)
    v2 = np.full((T, W), np.nan)
    
    if prev and Yf is None:
        Yf = Y.copy()
    
    a_t[0, :] = pars['mu'].flatten()
    P_t[0, :, :] = lyapunov(N, pars['phi'], Q)
    
    logLik = -0.5 * T * W * math.log(2 * math.pi)
    
    out = Kfilter(logLik, N, T, Y, pars['Z'], a_t, P_t, H, a_tt, P_tt, v2, v1, pars['phi'], pars['mu'], Q, prev, M, Yf, lik)
    return out

def ns_loadings(tau, lam):
    level = 1.0
    slope = (1 - np.exp(-lam * tau)) / (lam * tau) if tau > 1e-12 else 1.0
    curvature = slope - np.exp(-lam * tau)
    return np.array([level, slope, curvature])

def lyapunov(N, phi, Q):
    I = np.eye(N * N)
    K_mat = np.kron(phi, phi)
    vecP = np.linalg.solve(I - K_mat, Q.flatten())
    return vecP.reshape(N, N)

def Nelson_Siegel_factor_loadings(l, m):
    m = np.array(m)
    column1 = np.ones(len(m))
    column2 = (1 - np.exp(-l * m)) / (l * m)
    column3 = column2 - np.exp(-l * m)
    return np.column_stack((column1, column2, column3))

def compute_yield_curve(beta, lam, tau_array):
    return np.array([beta @ ns_loadings(t, lam) for t in tau_array])

def build_M(K, Sigma):
    SS = Sigma @ Sigma.T
    K_diag = np.array(K)
    out = np.zeros((3, 3))
    for i in range(3):
        for j in range(3):
            kij = K_diag[i] + K_diag[j]
            out[i, j] = (1 - np.exp(-kij)) / kij if kij > 1e-12 else 0.0
    SSout = SS * out
    M = np.linalg.cholesky(SSout)
    return M

def build_N_diag(LOT, a, b):
    return np.diag([LOT, a, b])

def sum_slope_curve(lam, LOT):
    s = 0.0
    for tau in range(1, LOT+1):
        s += ns_loadings(tau, lam)[1]
    return s

def sum_curvature_curve(lam, LOT):
    s = 0.0
    for tau in range(1, LOT+1):
        s += ns_loadings(tau, lam)[2]
    return s

def calc_theta(lam, LOT, M_, e1, e2):
    sum_h1 = 0.0
    sum_h2 = 0.0
    for tau in range(1, LOT+1):
        load = ns_loadings(tau, lam)
        h1 = load @ (M_ @ e1)
        h2 = load @ (M_ @ e2)
        sum_h1 += h1
        sum_h2 += h2
    if abs(sum_h1) < 1e-12:
        return np.pi/2 if sum_h2 > 0 else -np.pi/2
    return np.arctan(sum_h2/sum_h1)

def level_up_scenario(K, Sigma, lam, LOT):
    M_ = build_M(K, Sigma)
    a_ = sum_slope_curve(lam, LOT)
    b_ = sum_curvature_curve(lam, LOT)
    N_ = build_N_diag(LOT, a_, b_)
    NM = N_ @ M_
    NtN = NM @ NM.T
    eigvals, eigvecs = np.linalg.eig(NtN)
    idx = np.argsort(eigvals)[::-1]
    e1 = eigvecs[:, idx[0]]
    e2 = eigvecs[:, idx[1]]
    theta_ = calc_theta(lam, LOT, M_, e1, e2)
    Me1 = M_ @ e1
    Me2 = M_ @ e2
    sl_vec = np.cos(theta_) * Me1 + np.sin(theta_) * Me2
    load_LOT = ns_loadings(LOT, lam)
    shift_LOT = np.dot(sl_vec, load_LOT)
    s_ = 1.0 if shift_LOT >= 0 else -1.0
    return sl_vec, s_, theta_

def twist_scenario_parameters(K, Sigma, lam, LOT):
    
    M_ = build_M(K, Sigma)
    a_ = sum_slope_curve(lam, LOT)
    b_ = sum_curvature_curve(lam, LOT)
    N_ = build_N_diag(LOT, a_, b_)
    NM = N_ @ M_
    NtN = NM @ NM.T
    eigvals, eigvecs = np.linalg.eig(NtN)
    idx = np.argsort(eigvals)[::-1]
    e1 = eigvecs[:, idx[0]]
    e2 = eigvecs[:, idx[1]]
    theta_ = calc_theta(lam, LOT, M_, e1, e2)
    
    twist_vector = np.cos(theta_)*(M_ @ e2) - np.sin(theta_)*(M_ @ e1)
    return twist_vector, theta_


In [ ]:
# calibration utilizing previous lambda
def calibrate_model(train_data, prev_lambda=None, num_lambda=3):

    resultsall = []
    results = []
    
    
    if prev_lambda is None:
        lambdastep = initial_lambdastep.copy()
    else:
        lambdastep = np.linspace(prev_lambda - 0.02, prev_lambda + 0.02,5)
    
    for k in lambdastep:
        para0[0] = k
        print("Trying Lambda:", k)
        startTime = time.time()
        NoP = len(para0)
        
        lik0 = kalman(para=para0, Y=train_data, lik=True, prev=False, ahead=1, grid=grid)
        paraNext = para0.copy()
        likNext = lik0
        
        
        for j in range(1, maxstepn+1):
            if j % 1000 == 0:
                print("Iteration:", j)
            i = j % NoP
            paraP = paraNext.copy()
            paraM = paraNext.copy()
            paraP[i] = paraNext[i] + 10.0**(paraStep[i])
            paraM[i] = paraNext[i] - 10.0**(paraStep[i])
            
            likP = kalman(para=paraP, Y=train_data, lik=True, prev=False, ahead=1, grid=grid)
            likM = kalman(para=paraM, Y=train_data, lik=True, prev=False, ahead=1, grid=grid)
            
            if likP < likNext:
                paraNext = paraP.copy()
                likNext = likP
            elif likM < likNext:
                paraNext = paraM.copy()
                likNext = likM
            
            resultsall.append(np.concatenate(([j], paraNext, [likNext])))
            
            if j > NoP+1 and abs(resultsall[-1][gridn+15-1] - resultsall[-NoP][gridn+15-1]) < 1e-2: #j > NoP+1 and resultsall[-1][gridn+15-1] == resultsall[-NoP][gridn+15-1]:
                break
        
        results.append(np.concatenate(([j], paraNext, [likNext])))
        endTime = time.time()
        print("Lambda", k, "calibration time:", endTime - startTime)
    
    resultsall = np.array(resultsall)
    results = np.array(results)
    min_index = np.argmin(results[:, gridn+15-1])
    minresults_test = results[min_index, :]
    return minresults_test

In [ ]:
# calculate yield curves for ICS risk calculation
def calculate_yield_curves(para_opt, LSC_result, maturities):
    lambda_val = para_opt[0]
    K_est = (np.ones(3) - para_opt[13:16]) * tstep
    mu_est = para_opt[16:19] / 100
    
    Q_mat = np.zeros((3, 3))
    Q_mat[0, 0] = para_opt[19] * np.sqrt(tstep) / 100
    Q_mat[1, 0] = para_opt[20] * np.sqrt(tstep) / 100
    Q_mat[1, 1] = para_opt[21] * np.sqrt(tstep) / 100
    Q_mat[2, 0] = para_opt[22] * np.sqrt(tstep) / 100
    Q_mat[2, 1] = para_opt[23] * np.sqrt(tstep) / 100
    Q_mat[2, 2] = para_opt[24] * np.sqrt(tstep) / 100

    beta_calibrated = LSC_result['a_tt'][-1, :] / 100
    base_yields = compute_yield_curve(beta_calibrated, lambda_val, maturities)
    
    # Mean Reversion
    LOT = 30
    Delta = (1 - np.exp(-K_est)) * (mu_est - beta_calibrated)
    stressed_yields_mr = base_yields + np.array([np.dot(Delta, ns_loadings(t, lambda_val))
                                                 for t in maturities])
    
    # Level Up / Down
    sl_vector, s_sign, theta_val = level_up_scenario(K_est, Q_mat, lambda_val, LOT)
    n_inv = norm.ppf(0.995)
    shift_curve_level = np.array([s_sign * n_inv * np.dot(sl_vector, ns_loadings(t, lambda_val))
                                  for t in maturities])
    stressed_yields_level_up   = base_yields + shift_curve_level
    stressed_yields_level_down = base_yields - shift_curve_level

    # Twist
    twist_vector, theta_twist = twist_scenario_parameters(K_est, Q_mat, lambda_val, LOT)
    stressed_yields_twist_up   = base_yields + np.array([np.dot(twist_vector, ns_loadings(t, lambda_val))
                                                         for t in maturities])
    stressed_yields_twist_down = base_yields - np.array([np.dot(twist_vector, ns_loadings(t, lambda_val))
                                                         for t in maturities])
    yields_dict = {
        "base": base_yields,
        "mean_reversion": stressed_yields_mr,
        "level_up": stressed_yields_level_up,
        "level_down": stressed_yields_level_down,
        "twist_up": stressed_yields_twist_up,
        "twist_down": stressed_yields_twist_down,
    }
    return yields_dict

In [ ]:
def calibrate_params_no_lambda(train_data, fixed_lambda):
    """
    estimate parameters other than lambda
    """
    para = para0.copy()
    para[0] = fixed_lambda
    likNext = kalman(para=para, Y=train_data, lik=True, prev=False, ahead=1, grid=grid)
    
    NoP = len(para)   # 22
    lik_history = []
    
    for j in range(1, maxstepn+1):
        i = (j % (NoP-1)) + 1
        
        paraP = para.copy()
        paraM = para.copy()
        paraP[i] += 10.0**(paraStep[i])
        paraM[i] -= 10.0**(paraStep[i])
        
        likP = kalman(para=paraP, Y=train_data, lik=True, prev=False, ahead=1, grid=grid)
        likM = kalman(para=paraM, Y=train_data, lik=True, prev=False, ahead=1, grid=grid)
        
        if likP < likNext:
            para, likNext = paraP, likP
        elif likM < likNext:
            para, likNext = paraM, likM
        
        lik_history.append(likNext)
        if j > (NoP-1)+1 and abs(lik_history[-1] - lik_history[-(NoP-1)]) < 1e-2:
            break
    
    return para


def online_update_params(prev_para, train_data):
    para = prev_para.copy()
    likNext = kalman(para=para, Y=train_data, lik=True, prev=False, ahead=1, grid=grid)
    
    NoP = len(para)

    for i in range(1, NoP):
        paraP = para.copy()
        paraM = para.copy()
        paraP[i] += 10.0**(paraStep[i])
        paraM[i] -= 10.0**(paraStep[i])
        
        likP = kalman(para=paraP, Y=train_data, lik=True, prev=False, ahead=1, grid=grid)
        likM = kalman(para=paraM, Y=train_data, lik=True, prev=False, ahead=1, grid=grid)
        
        if likP < likNext:
            para, likNext = paraP, likP
        elif likM < likNext:
            para, likNext = paraM, likM
    
    return para


## Main calibration

In [ ]:

# load the data
data_y = pd.read_csv(wsdata + inputfile, header=0, index_col=0, encoding='cp1252')
data_y.index = pd.to_datetime(data_y.index)
data_y = data_y.resample("W-FRI").last().ffill()[maturities]

# set initial calibration period
init_start = '2010-01-01'
init_end   = '2018-03-31'
init_data  = data_y.loc[init_start:init_end].values * 100

print(f"Initial full calibration: {init_start} to {init_end}")

init_minres   = calibrate_model(init_data,
                                 prev_lambda=None,
                                 num_lambda=5)
prev_para     = init_minres[1:23].copy()
prev_lambda   = prev_para[0]


bt_start = pd.to_datetime(init_end) - pd.Timedelta(days=1)
data_bt   = data_y.loc[bt_start:]
dates     = data_bt.index
forecast_horizon = 1

quarterly_freq = 1 # mean reversion part seems to be too sensitive. set granular frequency
annual_freq    = 52

yield_curves = []
calibrated   = []
tstep = 52


BASE_UFR = 0.038
SW_PARAMS = dict(
    nrofcoup = 0,
    CRA      = 0,
    alfamin  = 0.05,
    Tau      = 1.0,
    T2       = 60,
)

SCENARIOS = [
    ("base",       lambda ufr: ufr),
    ("level_up",   lambda ufr: ufr * 1.10),
    ("level_down", lambda ufr: ufr * 0.90),
    ("mean_reversion", lambda ufr: ufr),
    ("twist_up",      lambda ufr: ufr),
    ("twist_down",    lambda ufr: ufr),
]

def sw_full_curve_from_ns(ns_yields, maturities, ufr):
    # ns_yields: list or array of 1..Max年のNSモデル金利(％)
    data_in = [[1, float(m), float(y)]
               for m, y in zip(maturities, ns_yields) if m <= 30]
    data_in = np.array(data_in)
    sw_res = smith_wilson_brute_force(
        Instrument="zero",
        data_in   = data_in,
        UFRac     = ufr,
        **SW_PARAMS
    )
    zeroac_pct = sw_res[:, 2]
    return {int(m): zeroac_pct[int(m)]
            for m in maturities if 0 < int(m) < len(zeroac_pct)}

yield_curves = []
for i in range(len(dates) - forecast_horizon):
    train_end = dates[i]
    train_data = data_y.loc[init_start:train_end].values * 100
    test_date  = dates[i + forecast_horizon]

    if i % annual_freq == 0:
        print(f"[{test_date.date()}] Annual full calibration")
        minres     = calibrate_model(train_data,
                                     prev_lambda=prev_lambda,
                                     num_lambda=5)
        para_opt   = minres[1:26].copy()
        prev_lambda = para_opt[0]
    elif i % quarterly_freq == 0:
        print(f"[{test_date.date()}] Quarterly calibration (λ={prev_lambda:.4f})")
        para_opt   = calibrate_params_no_lambda(train_data,
                                                fixed_lambda=prev_lambda)
    else:
        print(f"[{test_date.date()}] Weekly online update")
        para_opt   = online_update_params(prev_para, train_data)

    prev_para = para_opt.copy()
    calibrated.append(para_opt)

    lsc = kalman(para=para_opt,
                 Y=train_data,
                 lik=False,
                 prev=False,
                 ahead=forecast_horizon,
                 grid=grid)
    yields_ns = calculate_yield_curves(para_opt, lsc, maturities2_float)
    
    sw_results = {}
    for scen, ufr_func in SCENARIOS:
        ns30 = [y for (m,y) in zip(maturities2_float, yields_ns[scen]) if m <= 30]
        sw_results[scen] = sw_full_curve_from_ns(
            ns30,
            maturities2_float,
            ufr = ufr_func(BASE_UFR)
        )

    for j, m in enumerate(maturities2_float):
        mi = int(m)
        row = {
            'Test_Date': test_date,
            'Maturity':  m,
        }
        for scen, _ in SCENARIOS:
            key_name = {
                "base":            "Base Yield",
                "level_up":        "Level Up",
                "level_down":      "Level Down",
                "mean_reversion":  "Mean Reversion",
                "twist_up":        "Twist Up-to-Down",
                "twist_down":      "Twist Down-to-Up",
            }[scen]
            row[key_name] = sw_results[scen].get(mi, np.nan)
        yield_curves.append(row)

pd.DataFrame(yield_curves).to_csv("DNS30regu_ICS_1-year_risk_yields_v2.3.csv", index=False)
pd.DataFrame(calibrated).to_csv("DNS30regu_calibrated_params_v2.3.csv", index=False)


Initial full calibration: 2010-01-01 to 2018-03-31
Trying Lambda: 0.16
Lambda 0.16 calibration time: 11.57224178314209
[2018-04-06] Annual full calibration
Trying Lambda: 0.14
Lambda 0.14 calibration time: 0.03267788887023926
Trying Lambda: 0.15000000000000002
Lambda 0.15000000000000002 calibration time: 0.019816875457763672
Trying Lambda: 0.16
Lambda 0.16 calibration time: 11.689608097076416
Trying Lambda: 0.16999999999999998
Lambda 0.16999999999999998 calibration time: 5.8827009201049805
Trying Lambda: 0.18
Lambda 0.18 calibration time: 2.851409912109375
[2018-04-13] Quarterly calibration (λ=0.1800)
[2018-04-20] Quarterly calibration (λ=0.1800)
[2018-04-27] Quarterly calibration (λ=0.1800)
[2018-05-04] Quarterly calibration (λ=0.1800)
[2018-05-11] Quarterly calibration (λ=0.1800)
[2018-05-18] Quarterly calibration (λ=0.1800)
[2018-05-25] Quarterly calibration (λ=0.1800)
[2018-06-01] Quarterly calibration (λ=0.1800)
[2018-06-08] Quarterly calibration (λ=0.1800)
[2018-06-15] Quarterly 